### Data Analysis on MCI and ADD patient Data

Here a machine Learning Model to predict if the Patient has MCI, ADD or Not shall be buiild. A number of steps are going to be taken and this include;
1. Creating of the suitable Datasets from the three Excel files shared. The Datasets will include
a) MCI and Controls Dataset
b) ADD and Controls Dataset
c) MCI, ADD and controls Dataset

2. Pre-Analysis of the Data. We want to establish;
a) The number of features in the three datasets(excel files)
b) The Number of records in the datasets
c) The composition of the Dataset, how many MCI, ADD and controls records in the datasets
d) Are there any missing values?

3. Data Preprocessing
a) There were missing values in the Datasets. Some of the features had no values at all(100% missing values). Features with missing values greater than 80% were not considered for subsequent steps.
b) Records with null missing values whose features were 80% and above having values were dropped/removed from the datasets
c) Label Encoding was performed on the features. Machine learning only performs on encoded data(Converted to numbers). So performed encoding(Conversion of textual data into numbers). Once encoded, this data wil be used for building, training and testing of the Models

4. Model Building, Training and Testing 
 CLassification algorithms shall be used to build this models, Logistic Regression, SVM Adabost etc

5. Model Evaluation & Selection using Cross Validation, AUC etc



In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(45)
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pandas.plotting import table
from scipy import interp

from sklearn.feature_selection import SelectKBest, f_classif,chi2
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score,log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer

from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn.model_selection import KFold,cross_val_score,GridSearchCV

from sklearn.metrics import accuracy_score, roc_auc_score,confusion_matrix
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns

from sklearn.metrics import recall_score, precision_score,f1_score
from skfeature.function.similarity_based import fisher_score   
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

1. Helper Functions that will come in Handy

In [2]:
# Create table for missing data analysis
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)*100
    missing_data = pd.concat([total, percent], axis=1, keys=['Total missing Values', '% of Missing Values'])
    return missing_data

#pick the Top 10 Features by Default
def view_best_features(features, labels, number_of_features = 10):
    selector = SelectKBest(score_func=f_classif, k = number_of_features) #f_classif
    #New dataframe with the selected features for later use in the classifier. fit() method works too, if you want only the feature names and their corresponding scores
   
    X_new = selector.fit_transform(features, labels)
    names = features.columns.values[selector.get_support()]
    scores = selector.scores_[selector.get_support()]

    names_scores = list(zip(names, scores))
    ns_df = pd.DataFrame(data = names_scores, columns=['Feature', 'Scores'])
    #Sort the dataframe for better visualization
    ns_df_sorted = ns_df.sort_values('Scores',ascending=False)
    return ns_df_sorted


def drop_null_columns(data, percentage):
     return data[data.columns[data.isnull().mean() < percentage]]

In [3]:
def load_data(file_path):
    return pd.read_excel(file_path) 

#We are adding a column, label to indicate if patient has (MCI or ADD) or not(Control)
def label_data(data, label):
    data['label'] = label
    return data  
#fixing messy column names
def fixing_column_names(data):
    # remove spaces in columns name
    data.columns = data.columns.str.replace(' ','_')
    # remove xters like ():[] in the column names
    data.columns = data.columns.str.replace("[()\[\]:]",'')
    # convert to lower case
    data.columns = data.columns.str.lower()
    return data
def combine_two_dataframes(first_dataframe, second_dataframe):
    return pd.concat([first_dataframe, second_dataframe], ignore_index=True, sort=False)

def encode_data(data):
        # Categorical boolean mask
    categorical_feature_mask = data.dtypes==object
    # filter categorical columns using mask and turn it into a list
    categorical_cols = data.columns[categorical_feature_mask].tolist()
    # import labelencoder
    from sklearn.preprocessing import LabelEncoder
    # instantiate labelencoder object
    le = LabelEncoder()
   # apply le on categorical feature columns
    data[categorical_cols] = data[categorical_cols].apply(lambda col: le.fit_transform(col))
    return data
def scale_training_set(X_train, X_test):
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train = scaling.transform(X_train)
    X_test = scaling.transform(X_test)
    return X_train, X_test

1. Loading dataset as is

In [4]:
data_mci = load_data("data/AD_PAGN_GN_C01_00121_MCI.xlsx")
data_controls = load_data("data/AD_PAGN_GN_C01_00141_control.xlsx")
data_add         = load_data("data/AD_PAGN_GN_C01_00240_AD.xlsx")


2. Observing the first two records of the mci data

In [5]:
data_mci.head(2)

,Recording timestamp,Project name,Export date,Participant name,Gender,Glasses,Age,Recording name,Recording date,Recording start time,Recording duration,Timeline name,Recording Fixation filter name,Recording software version,Recording resolution height,Recording resolution width,Recording monitor latency,Average calibration accuracy (mm),Average calibration precision SD (mm),Average calibration precision RMS (mm),Average calibration accuracy (degrees),Average calibration precision SD (degrees),Average calibration precision RMS (degrees),Average calibration accuracy (pixels),Average calibration precision SD (pixels),Average calibration precision RMS (pixels),Eyetracker timestamp,Event,Event value,Gaze point X,Gaze point Y,Gaze point left X,Gaze point left Y,Gaze point right X,Gaze point right Y,Gaze direction left X,Gaze direction left Y,Gaze direction left Z,Gaze direction right X,Gaze direction right Y,Gaze direction right Z,Pupil diameter left,Pupil diameter right,Validity left,Validity right,Eye position left X (DACSmm),Eye position left Y (DACSmm),Eye position left Z (DACSmm),Eye position right X (DACSmm),Eye position right Y (DACSmm),Eye position right Z (DACSmm),Gaze point left X (DACSmm),Gaze point left Y (DACSmm),Gaze point right X (DACSmm),Gaze point right Y (DACSmm),Gaze point X (MCSnorm),Gaze point Y (MCSnorm),Gaze point left X (MCSnorm),Gaze point left Y (MCSnorm),Gaze point right X (MCSnorm),Gaze point right Y (MCSnorm),Presented Stimulus name,Presented Media name,Presented Media width,Presented Media height,Presented Media position X (DACSpx),Presented Media position Y (DACSpx),Original Media width,Original Media height,Eye movement type,Gaze event duration,Eye movement type index,Fixation point X,Fixation point Y,Fixation point X (MCSnorm),Fixation point Y (MCSnorm),AOI hit [TP_Lt - TP_Lt_C],AOI hit [TP_Lt - TP_Lt_L],AOI hit [TP_Lt - TP_Lt_R],AOI hit [TP_Rt - TP_Rt_C],AOI hit [TP_Rt - TP_Rt_L],AOI hit [TP_Rt - TP_Rt_R],AOI hit [CueG - CueG_C],AOI hit [CueG - CueG_L],AOI hit [CueG - CueG_R],AOI hit [TG_Rt - TG_Rt_C],AOI hit [TG_Rt - TG_Rt_L],AOI hit [TG_Rt - TG_Rt_R],AOI hit [CueP - CueP_C],AOI hit [CueP - CueP_L],AOI hit [CueP - CueP_R],AOI hit [TN_Rt - TN_Rt_C],AOI hit [TN_Rt - TN_Rt_L],AOI hit [TN_Rt - TN_Rt_R],AOI hit [FixG - FixG],AOI hit [TN_Lt - TN_Lt_C],AOI hit [TN_Lt - TN_Lt_L],AOI hit [TN_Lt - TN_Lt_R],AOI hit [TG_Lt - TG_Lt_C],AOI hit [TG_Lt - TG_Lt_L],AOI hit [TG_Lt - TG_Lt_R],AOI hit [FixN - FixN],AOI hit [CueN - CueN_C],AOI hit [CueN - CueN_L],AOI hit [CueN - CueN_R],AOI hit [TA_Rt - TA_Rt_C],AOI hit [TA_Rt - TA_Rt_L],AOI hit [TA_Rt - TA_Rt_R],AOI hit [CueA - CueA_C],AOI hit [CueA - CueA_L],AOI hit [CueA - CueA_R],AOI hit [FixP - FixP],AOI hit [TA_Lt - TA_Lt_C],AOI hit [TA_Lt - TA_Lt_L],AOI hit [TA_Lt - TA_Lt_R],AOI hit [FixA - FixA],AOI hit [Target:Correct],AOI hit [Target:Uncorrect],Client area position X (DACSpx),Client area position Y (DACSpx),Viewport position X,Viewport position Y,Viewport width,Viewport height,Full page width,Full page height,Mouse position X,Mouse position Y
0,0,AD_PAGN,03/05/2020 09:28:24,C01_121,Female,Yes,≥71,GN_C01_121,07/08/2019 13:22:56,13:22:56.310,323753,GNsession,Raw,1.118.21001,1080,1920,10,4,2.7,2,0.34,0.22,0.16,15,10,7,NaN,RecordingStart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,82,AD_PAGN,03/05/2020 09:28:24,C01_121,Female,Yes,≥71,GN_C01_121,07/08/2019 13:22:56,13:22:56.310,323753,GNsession,Raw,1.118.21001,1080,1920,10,4,2.7,2,0.34,0.22,0.16,15,10,7,NaN,Eye tracker TTL in,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

3. Ensuring the the column names are not messy

In [6]:
data_mci_fixed_columns       =fixing_column_names(data_mci)
data_controls_fixed_columns      =fixing_column_names(data_controls)
data_add_fixed_columns      =fixing_column_names(data_add)

In [7]:
data_mci_fixed_columns.head(5)

,recording_timestamp,project_name,export_date,participant_name,gender,glasses,age,recording_name,recording_date,recording_start_time,recording_duration,timeline_name,recording_fixation_filter_name,recording_software_version,recording_resolution_height,recording_resolution_width,recording_monitor_latency,average_calibration_accuracy_mm,average_calibration_precision_sd_mm,average_calibration_precision_rms_mm,average_calibration_accuracy_degrees,average_calibration_precision_sd_degrees,average_calibration_precision_rms_degrees,average_calibration_accuracy_pixels,average_calibration_precision_sd_pixels,average_calibration_precision_rms_pixels,eyetracker_timestamp,event,event_value,gaze_point_x,gaze_point_y,gaze_point_left_x,gaze_point_left_y,gaze_point_right_x,gaze_point_right_y,gaze_direction_left_x,gaze_direction_left_y,gaze_direction_left_z,gaze_direction_right_x,gaze_direction_right_y,gaze_direction_right_z,pupil_diameter_left,pupil_diameter_right,validity_left,validity_right,eye_position_left_x_dacsmm,eye_position_left_y_dacsmm,eye_position_left_z_dacsmm,eye_position_right_x_dacsmm,eye_position_right_y_dacsmm,eye_position_right_z_dacsmm,gaze_point_left_x_dacsmm,gaze_point_left_y_dacsmm,gaze_point_right_x_dacsmm,gaze_point_right_y_dacsmm,gaze_point_x_mcsnorm,gaze_point_y_mcsnorm,gaze_point_left_x_mcsnorm,gaze_point_left_y_mcsnorm,gaze_point_right_x_mcsnorm,gaze_point_right_y_mcsnorm,presented_stimulus_name,presented_media_name,presented_media_width,presented_media_height,presented_media_position_x_dacspx,presented_media_position_y_dacspx,original_media_width,original_media_height,eye_movement_type,gaze_event_duration,eye_movement_type_index,fixation_point_x,fixation_point_y,fixation_point_x_mcsnorm,fixation_point_y_mcsnorm,aoi_hit_tp_lt_-_tp_lt_c,aoi_hit_tp_lt_-_tp_lt_l,aoi_hit_tp_lt_-_tp_lt_r,aoi_hit_tp_rt_-_tp_rt_c,aoi_hit_tp_rt_-_tp_rt_l,aoi_hit_tp_rt_-_tp_rt_r,aoi_hit_cueg_-_cueg_c,aoi_hit_cueg_-_cueg_l,aoi_hit_cueg_-_cueg_r,aoi_hit_tg_rt_-_tg_rt_c,aoi_hit_tg_rt_-_tg_rt_l,aoi_hit_tg_rt_-_tg_rt_r,aoi_hit_cuep_-_cuep_c,aoi_hit_cuep_-_cuep_l,aoi_hit_cuep_-_cuep_r,aoi_hit_tn_rt_-_tn_rt_c,aoi_hit_tn_rt_-_tn_rt_l,aoi_hit_tn_rt_-_tn_rt_r,aoi_hit_fixg_-_fixg,aoi_hit_tn_lt_-_tn_lt_c,aoi_hit_tn_lt_-_tn_lt_l,aoi_hit_tn_lt_-_tn_lt_r,aoi_hit_tg_lt_-_tg_lt_c,aoi_hit_tg_lt_-_tg_lt_l,aoi_hit_tg_lt_-_tg_lt_r,aoi_hit_fixn_-_fixn,aoi_hit_cuen_-_cuen_c,aoi_hit_cuen_-_cuen_l,aoi_hit_cuen_-_cuen_r,aoi_hit_ta_rt_-_ta_rt_c,aoi_hit_ta_rt_-_ta_rt_l,aoi_hit_ta_rt_-_ta_rt_r,aoi_hit_cuea_-_cuea_c,aoi_hit_cuea_-_cuea_l,aoi_hit_cuea_-_cuea_r,aoi_hit_fixp_-_fixp,aoi_hit_ta_lt_-_ta_lt_c,aoi_hit_ta_lt_-_ta_lt_l,aoi_hit_ta_lt_-_ta_lt_r,aoi_hit_fixa_-_fixa,aoi_hit_targetcorrect,aoi_hit_targetuncorrect,client_area_position_x_dacspx,client_area_position_y_dacspx,viewport_position_x,viewport_position_y,viewport_width,viewport_height,full_page_width,full_page_height,mouse_position_x,mouse_position_y
0,0,AD_PAGN,03/05/2020 09:28:24,C01_121,Female,Yes,≥71,GN_C01_121,07/08/2019 13:22:56,13:22:56.310,323753,GNsession,Raw,1.118.21001,1080,1920,10,4,2.7,2,0.34,0.22,0.16,15,10,7,NaN,RecordingStart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,82,AD_PAGN,03/05/2020 09:28:24,C01_121,Female,Yes,≥71,GN_C01_121,07/08/2019 13:22:56,13:22:56.310,323753,GNsession,Raw,1.118.21001,1080,1920,10,4,2.7,2,0.34,0.22,0.16,15,10,7,NaN,Eye tracker TTL in,255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

4. Label the Data, for subsequent classification algorithm development: Mark Patients with MCI as 1 and those with ADD as -1 and those without Dimentia ie controls: 0

In [8]:
data_mci_labelled       =label_data(data_mci_fixed_columns, 1)
data_controls_labelled      =label_data(data_controls_fixed_columns, 0)
data_add_labelled      =label_data(data_add_fixed_columns,-1)

In [9]:
data_mci_labelled.describe()

,recording_timestamp,recording_duration,recording_resolution_height,recording_resolution_width,recording_monitor_latency,average_calibration_accuracy_mm,average_calibration_precision_sd_mm,average_calibration_precision_rms_mm,average_calibration_accuracy_degrees,average_calibration_precision_sd_degrees,average_calibration_precision_rms_degrees,average_calibration_accuracy_pixels,average_calibration_precision_sd_pixels,average_calibration_precision_rms_pixels,eyetracker_timestamp,gaze_point_x,gaze_point_y,gaze_point_left_x,gaze_point_left_y,gaze_point_right_x,gaze_point_right_y,gaze_direction_left_x,gaze_direction_left_y,gaze_direction_left_z,gaze_direction_right_x,gaze_direction_right_y,gaze_direction_right_z,pupil_diameter_left,pupil_diameter_right,eye_position_left_x_dacsmm,eye_position_left_y_dacsmm,eye_position_left_z_dacsmm,eye_position_right_x_dacsmm,eye_position_right_y_dacsmm,eye_position_right_z_dacsmm,gaze_point_left_x_dacsmm,gaze_point_left_y_dacsmm,gaze_point_right_x_dacsmm,gaze_point_right_y_dacsmm,gaze_point_x_mcsnorm,gaze_point_y_mcsnorm,gaze_point_left_x_mcsnorm,gaze_point_left_y_mcsnorm,gaze_point_right_x_mcsnorm,gaze_point_right_y_mcsnorm,presented_media_width,presented_media_height,presented_media_position_x_dacspx,presented_media_position_y_dacspx,original_media_width,original_media_height,gaze_event_duration,eye_movement_type_index,fixation_point_x,fixation_point_y,fixation_point_x_mcsnorm,fixation_point_y_mcsnorm,aoi_hit_tp_lt_-_tp_lt_c,aoi_hit_tp_lt_-_tp_lt_l,aoi_hit_tp_lt_-_tp_lt_r,aoi_hit_tp_rt_-_tp_rt_c,aoi_hit_tp_rt_-_tp_rt_l,aoi_hit_tp_rt_-_tp_rt_r,aoi_hit_cueg_-_cueg_c,aoi_hit_cueg_-_cueg_l,aoi_hit_cueg_-_cueg_r,aoi_hit_tg_rt_-_tg_rt_c,aoi_hit_tg_rt_-_tg_rt_l,aoi_hit_tg_rt_-_tg_rt_r,aoi_hit_cuep_-_cuep_c,aoi_hit_cuep_-_cuep_l,aoi_hit_cuep_-_cuep_r,aoi_hit_tn_rt_-_tn_rt_c,aoi_hit_tn_rt_-_tn_rt_l,aoi_hit_tn_rt_-_tn_rt_r,aoi_hit_fixg_-_fixg,aoi_hit_tn_lt_-_tn_lt_c,aoi_hit_tn_lt_-_tn_lt_l,aoi_hit_tn_lt_-_tn_lt_r,aoi_hit_tg_lt_-_tg_lt_c,aoi_hit_tg_lt_-_tg_lt_l,aoi_hit_tg_lt_-_tg_lt_r,aoi_hit_fixn_-_fixn,aoi_hit_cuen_-_cuen_c,aoi_hit_cuen_-_cuen_l,aoi_hit_cuen_-_cuen_r,aoi_hit_ta_rt_-_ta_rt_c,aoi_hit_ta_rt_-_ta_rt_l,aoi_hit_ta_rt_-_ta_rt_r,aoi_hit_cuea_-_cuea_c,aoi_hit_cuea_-_cuea_l,aoi_hit_cuea_-_cuea_r,aoi_hit_fixp_-_fixp,aoi_hit_ta_lt_-_ta_lt_c,aoi_hit_ta_lt_-_ta_lt_l,aoi_hit_ta_lt_-_ta_lt_r,aoi_hit_fixa_-_fixa,aoi_hit_targetcorrect,aoi_hit_targetuncorrect,client_area_position_x_dacspx,client_area_position_y_dacspx,viewport_position_x,viewport_position_y,viewport_width,viewport_height,full_page_width,full_page_height,mouse_position_x,mouse_position_y,label
count,97889.000000,97889.0,97889.0,97889.0,97889.0,97889.0,9.788900e+04,97889.0,9.788900e+04,9.788900e+04,9.788900e+04,97889.0,97889.0,97889.0,9.697100e+04,91606.000000,91606.000000,91527.000000,91527.000000,91252.000000,91252.000000,91527.000000,91527.000000,91527.000000,91252.000000,91252.000000,91252.000000,91527.000000,91252.000000,91527.000000,91527.000000,91527.000000,91252.000000,91252.000000,91252.000000,91527.000000,91527.000000,91252.000000,91252.000000,79531.000000,79531.000000,79493.000000,79493.000000,79233.000000,79233.000000,84391.0,84391.0,84391.0,84391.0,84391.0,84391.0,97885.000000,97885.000000,92466.000000,92466.000000,80250.000000,80250.000000,0.0,0.0,0.0,0.0,0.0,0.0,14539.000000,14539.000000,14539.000000,9954.000000,9954.000000,9954.000000,0.0,0.0,0.0,7239.000000,7239.0,7239.000000,9148.000000,6341.000000,6341.000000,6341.0,17194.000000,17194.000000,17194.000000,4574.000000,7277.000000,7277.0,7277.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97889.000000,97889.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.000000,184.000000,97889.0
mean,161905.446026,323753.0,1080.0,1920.0,10.0,4.0,2.700000e+00,2.0,3.400000e-01,2.200000e-01,1.600000e-01,15.0,10.0,7.0,1.387310e+09,919.683263,532.670524,916.759557,533.107793,924.468472,532.836562,0.044621,0.031374,-0.989673,-0.042487,0.032156,-0.990599,3.341274,2.876273,223.317975,126.829297,635.6700

# Understanding the Dataset

Establishing the extent of Missing values in the Datasets. We may drop features with 100% missing values for the start

1. Dealing with Mild Cognitive Impairment(MCI) data
A number of features in the data have 100% missing values. We have thus dropped them.

In [10]:
draw_missing_data_table(data_mci_labelled)

,Total missing Values,% of Missing Values
aoi_hit_tp_rt_-_tp_rt_l,97889,100.000000
aoi_hit_cuep_-_cuep_r,97889,100.000000
aoi_hit_cuep_-_cuep_c,97889,100.000000
aoi_hit_tp_rt_-_tp_rt_r,97889,100.000000
aoi_hit_tp_rt_-_tp_rt_c,97889,100.000000
aoi_hit_ta_rt_-_ta_rt_c,97889,100.000000
aoi_hit_ta_rt_-_ta_rt_l,97889,100.000000
aoi_hit_ta_rt_-_ta_rt_r,97889,100.000000
aoi_hit_cuea_-_cuea_c,97889,100.000000
aoi_hit_cuea_-_cuea_l,97889,100.000000


2. View Missing values of the Controls Dataset

In [11]:
draw_missing_data_table(data_controls_labelled)

,Total missing Values,% of Missing Values
aoi_hit_cuea_-_cuea_c,96554,100.000000
aoi_hit_ta_lt_-_ta_lt_r,96554,100.000000
aoi_hit_ta_rt_-_ta_rt_l,96554,100.000000
aoi_hit_tp_lt_-_tp_lt_r,96554,100.000000
aoi_hit_tp_lt_-_tp_lt_l,96554,100.000000
aoi_hit_tp_lt_-_tp_lt_c,96554,100.000000
aoi_hit_cuea_-_cuea_r,96554,100.000000
aoi_hit_cuea_-_cuea_l,96554,100.000000
aoi_hit_ta_rt_-_ta_rt_r,96554,100.000000
aoi_hit_ta_lt_-_ta_lt_l,96554,100.000000


3. View missing value of the ADD Dataset

In [12]:
draw_missing_data_table(data_add_labelled)

,Total missing Values,% of Missing Values
aoi_hit_cuea_-_cuea_c,94734,100.000000
aoi_hit_tp_lt_-_tp_lt_c,94734,100.000000
aoi_hit_tp_lt_-_tp_lt_r,94734,100.000000
aoi_hit_cuea_-_cuea_r,94734,100.000000
aoi_hit_cuea_-_cuea_l,94734,100.000000
aoi_hit_ta_lt_-_ta_lt_r,94734,100.000000
aoi_hit_ta_lt_-_ta_lt_l,94734,100.000000
aoi_hit_ta_lt_-_ta_lt_c,94734,100.000000
aoi_hit_tp_rt_-_tp_rt_r,94734,100.000000
aoi_hit_fixa_-_fixa,94734,100.000000


In [13]:
#Filter out the features with more than 100% NULL values and then drop those columns from the DataFrame
data_mci_with_minimal_missing_values = drop_null_columns(data_mci_labelled,0.8)
data_controls_with_minimal_missing_values = drop_null_columns(data_controls_labelled,0.8)
data_add_with_minimal_missing_values = drop_null_columns(data_add_labelled,0.8)

In [14]:
draw_missing_data_table(data_mci_with_minimal_missing_values)

,Total missing Values,% of Missing Values
gaze_point_right_x_mcsnorm,18656,19.058321
gaze_point_right_y_mcsnorm,18656,19.058321
gaze_point_left_x_mcsnorm,18396,18.792714
gaze_point_left_y_mcsnorm,18396,18.792714
gaze_point_x_mcsnorm,18358,18.753895
gaze_point_y_mcsnorm,18358,18.753895
fixation_point_y_mcsnorm,17639,18.019389
fixation_point_x_mcsnorm,17639,18.019389
presented_media_position_y_dacspx,13498,13.789088
presented_media_width,13498,13.789088


Let's Create datasets that willl be used for development of classifcation models. We'll datasets comprising of;
1. MCI and Controls labelled 1 and 0 respectively
2. ADD and Controls labelled -1 and 0 respectively
3. A dataset containing MCI, ADD and controls labelled 1, 0 and -1 respectively

In [15]:
mci_controls_data = combine_two_dataframes(data_mci_with_minimal_missing_values, data_controls_with_minimal_missing_values)
add_controls_data = combine_two_dataframes(data_add_with_minimal_missing_values, data_controls_with_minimal_missing_values)
mci_add_controls_data = combine_two_dataframes(mci_controls_data, data_add_with_minimal_missing_values)

# Data PreProcessing

1. Performing Encoding of the Features

# Feature Selection

Now that we have the Data, we'll have to perform the following;
1. Select the best performing Features
2. Perform Some Cross Validation
3. Build, Train and Test Our Models off the Data
4. Choose the best performing model

In [16]:
mci_controls_data.tail(4)

,recording_timestamp,project_name,export_date,participant_name,gender,glasses,age,recording_name,recording_date,recording_start_time,recording_duration,timeline_name,recording_fixation_filter_name,recording_software_version,recording_resolution_height,recording_resolution_width,recording_monitor_latency,average_calibration_accuracy_mm,average_calibration_precision_sd_mm,average_calibration_precision_rms_mm,average_calibration_accuracy_degrees,average_calibration_precision_sd_degrees,average_calibration_precision_rms_degrees,average_calibration_accuracy_pixels,average_calibration_precision_sd_pixels,average_calibration_precision_rms_pixels,eyetracker_timestamp,gaze_point_x,gaze_point_y,gaze_point_left_x,gaze_point_left_y,gaze_point_right_x,gaze_point_right_y,gaze_direction_left_x,gaze_direction_left_y,gaze_direction_left_z,gaze_direction_right_x,gaze_direction_right_y,gaze_direction_right_z,pupil_diameter_left,pupil_diameter_right,validity_left,validity_right,eye_position_left_x_dacsmm,eye_position_left_y_dacsmm,eye_position_left_z_dacsmm,eye_position_right_x_dacsmm,eye_position_right_y_dacsmm,eye_position_right_z_dacsmm,gaze_point_left_x_dacsmm,gaze_point_left_y_dacsmm,gaze_point_right_x_dacsmm,gaze_point_right_y_dacsmm,gaze_point_x_mcsnorm,gaze_point_y_mcsnorm,gaze_point_left_x_mcsnorm,gaze_point_left_y_mcsnorm,gaze_point_right_x_mcsnorm,gaze_point_right_y_mcsnorm,presented_stimulus_name,presented_media_name,presented_media_width,presented_media_height,presented_media_position_x_dacspx,presented_media_position_y_dacspx,original_media_width,original_media_height,eye_movement_type,gaze_event_duration,eye_movement_type_index,fixation_point_x,fixation_point_y,fixation_point_x_mcsnorm,fixation_point_y_mcsnorm,aoi_hit_targetcorrect,aoi_hit_targetuncorrect,label
194439,319140,AD_PAGN,03/11/2020 19:11:03,C01_00141,Female,No,≥70,GN_C01_00141,01/28/2020 15:38:16,15:38:16.120,319321,GNsession,Raw,1.118.21001,1080,1920,10,0.4,1.3,1.1,0.04,0.11,0.1,2,5,4,8.827743e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invalid,Invalid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GoodBye,GoodBye.BMP,1920.0,1080.0,0.0,0.0,1920.0,1080.0,EyesNotFound,3.0,840.0,NaN,NaN,NaN,NaN,0,0,0
194440,319143,AD_PAGN,03/11/2020 19:11:03,C01_00141,Female,No,≥70,GN_C01_00141,01/28/2020 15:38:16,15:38:16.120,319321,GNsession,Raw,1.118.21001,1080,1920,10,0.4,1.3,1.1,0.04,0.11,0.1,2,5,4,8.827746e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Invalid,Invalid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GoodBye,GoodBye.BMP,1920.0,1080.0,0.0,0.0,1920.0,1080.0,EyesNotFound,3.0,841.0,NaN,NaN,NaN,NaN,0,0,0
194441,319160,AD_PAGN,03/11/2020 19:11:03,C01_00141,Female,No,≥70,GN_C01_00141,01/28/2020 15:38:16,15:38:16.120,319321,GNsession,Raw,1.118.21001,1080,1920,10,0.4,1.3,1.1,0.04,0.11,0.1,2,5,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
194442,319321,AD_PAGN,03/11/2020 19:11:03,C01_00141,Female,No,≥70,GN_C01_00141,01/28/2020 15:38:16,15:38:16.120,319321,GNsession,Raw,1.118.21001,1080,1920,10,0.4,1.3,1.1,0.04,0.11,0.1,2,5,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [17]:
mci_controls_data.columns

Index(['recording_timestamp', 'project_name', 'export_date', 'participant_name', 'gender', 'glasses', 'age', 'recording_name', 'recording_date', 'recording_start_time', 'recording_duration', 'timeline_name', 'recording_fixation_filter_name', 'recording_software_version', 'recording_resolution_height', 'recording_resolution_width', 'recording_monitor_latency', 'average_calibration_accuracy_mm', 'average_calibration_precision_sd_mm', 'average_calibration_precision_rms_mm', 'average_calibration_accuracy_degrees', 'average_calibration_precision_sd_degrees', 'average_calibration_precision_rms_degrees', 'average_calibration_accuracy_pixels', 'average_calibration_precision_sd_pixels', 'average_calibration_precision_rms_pixels', 'eyetracker_timestamp', 'gaze_point_x', 'gaze_point_y', 'gaze_point_left_x', 'gaze_point_left_y', 'gaze_point_right_x', 'gaze_point_right_y', 'gaze_direction_left_x', 'gaze_direction_left_y', 'gaze_direction_left_z', 'gaze_direction_right_x', 'gaze_direction_right_y',


We have considered the features below for builing of the Model
eyetracker_timestamp', 'gaze_point_x', 'gaze_point_y', 'gaze_point_left_x','gaze_point_left_y','gaze_point_right_x', 'gaze_point_right_y', 'gaze_direction_left_x', 'gaze_direction_left_y', 'gaze_direction_left_z', 'gaze_direction_right_x', 'gaze_direction_right_y','gaze_direction_right_z', 'pupil_diameter_left', 'pupil_diameter_right', 'validity_left', 'validity_right',  'fixation_point_x', 'fixation_point_y', 'label'

From Observation, the features that capture the configurations on the application 
capturing the data eg recording_software_version were dropped

In [18]:
mci_controls_data_selected_features= mci_controls_data[['eyetracker_timestamp', 'gaze_point_x', 'gaze_point_y', 'gaze_point_left_x', 'gaze_point_left_y', 'gaze_point_right_x', 
                                                             'gaze_point_right_y', 'gaze_direction_left_x', 'gaze_direction_left_y', 'gaze_direction_left_z', 'gaze_direction_right_x', 'gaze_direction_right_y',
       'gaze_direction_right_z', 'pupil_diameter_left', 'pupil_diameter_right', 'validity_left', 'validity_right',  'fixation_point_x', 'fixation_point_y', 'label'
                                                             ]]

In [19]:
add_controls_data_selected_features= add_controls_data[['eyetracker_timestamp', 'gaze_point_x', 'gaze_point_y', 'gaze_point_left_x', 'gaze_point_left_y', 'gaze_point_right_x', 
                                                             'gaze_point_right_y', 'gaze_direction_left_x', 'gaze_direction_left_y', 'gaze_direction_left_z', 'gaze_direction_right_x', 'gaze_direction_right_y',
       'gaze_direction_right_z', 'pupil_diameter_left', 'pupil_diameter_right', 'validity_left', 'validity_right',  'fixation_point_x', 'fixation_point_y', 'label'
                                                             ]]

In [20]:
mci_add_controls_data_selected_features= mci_add_controls_data[['eyetracker_timestamp', 'gaze_point_x', 'gaze_point_y', 'gaze_point_left_x', 'gaze_point_left_y', 'gaze_point_right_x', 
                                                             'gaze_point_right_y', 'gaze_direction_left_x', 'gaze_direction_left_y', 'gaze_direction_left_z', 'gaze_direction_right_x', 'gaze_direction_right_y',
       'gaze_direction_right_z', 'pupil_diameter_left', 'pupil_diameter_right', 'validity_left', 'validity_right',  'fixation_point_x', 'fixation_point_y', 'label'
                                                             ]]

In [21]:
#Let's remove the Null Records from the Data. 
mci_controls_data_selected = mci_controls_data_selected_features.dropna()
add_controls_data_selected = add_controls_data_selected_features.dropna()
mci_add_controls_data_selected = mci_add_controls_data_selected_features.dropna()

Lets encode the Data. Convert Any text based values in the data into digits. For the ML algorithms to use

In [22]:
mci_controls_encoded = encode_data(mci_controls_data_selected)
add_controls_encoded = encode_data(add_controls_data_selected)
mci_add_controls_encoded = encode_data(mci_add_controls_data_selected)

Separating the Features(Independent Variables) from the Labels(Dependent/Target) Variable

In [23]:
#Dataset with MCI + Controls
mci_controls_features = mci_controls_encoded.drop(['label'], axis=1)
mci_controls_label = mci_controls_encoded['label']

#ADD + Controls
add_controls_features = add_controls_encoded.drop(['label'], axis=1)
add_controls_label = add_controls_encoded['label']

#MCI+ADD+Controls
mci_add_controls_features = mci_add_controls_encoded.drop(['label'], axis=1)
mci_add_controls_label = mci_add_controls_encoded['label']


In [24]:
mci_controls_features.columns

Index(['eyetracker_timestamp', 'gaze_point_x', 'gaze_point_y', 'gaze_point_left_x', 'gaze_point_left_y', 'gaze_point_right_x', 'gaze_point_right_y', 'gaze_direction_left_x', 'gaze_direction_left_y', 'gaze_direction_left_z', 'gaze_direction_right_x', 'gaze_direction_right_y', 'gaze_direction_right_z', 'pupil_diameter_left', 'pupil_diameter_right', 'validity_left', 'validity_right', 'fixation_point_x', 'fixation_point_y'], dtype='object')

# Feature Selection

Selecting the Right Features/Variables for the Model Training

I have used univariate measures(Specifically the, Analysis of Variance (ANOVA) statistical test) to select a variable based on its level of association with its target(MCI, AD or Control), 
the class SelectPercentile provides an automatic procedure 
for keeping only a certain percentage of the best, associated features.

The order of relevance of the variables/features to the target is eyetracker_timestamp , pupil_diameter_left,	pupil_diameter_right ,gaze_direction_right_y,	gaze_direction_left_y, gaze_direction_right_z, gaze_direction_left_z, gaze_point_right_y, gaze_point_y,	fixation_point_y, gaze_point_left_y, gaze_point_right_x, gaze_direction_right_x , gaze_point_x and	fixation_point_x respectively with eyetracker_timestamp the most relevant and fixation_point_x least relevant

In [25]:
#I have started by picking the 10 To Performing Features, this can be view in the next Cells
mci_controls_best_features = view_best_features(mci_controls_features,mci_controls_label,10)
add_controls_best_features = view_best_features(add_controls_features,add_controls_label,10)
mci_add_controls_best_features = view_best_features(mci_add_controls_features,mci_add_controls_label,10)

/home/pesapal030/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [15 16] are constant.
  UserWarning)
/home/pesapal030/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [15 16] are constant.
  UserWarning)
/home/pesapal030/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [15 16] are constant.
  UserWarning)


In [26]:
mci_controls_best_features.head(10)

,Feature,Scores
0,eyetracker_timestamp,2.880726e+08
7,pupil_diameter_left,1.001376e+06
8,pupil_diameter_right,3.905918e+04
5,gaze_direction_right_y,6.773853e+03
3,gaze_direction_left_y,6.647441e+03
6,gaze_direction_right_z,8.670755e+02
4,gaze_direction_left_z,7.455610e+02
2,gaze_point_right_y,4.993661e+02
1,gaze_point_y,4.827611e+02
9,fixation_point_y,4.827611e+02


# Training of Model

1. Using MCI Data

In [27]:
X_train, X_test, y_train, y_test = train_test_split(mci_controls_features, mci_controls_label, test_size=0.3, random_state=40)

In [28]:
X_train, X_test = scale_training_set(X_train, X_test)

/home/pesapal030/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


1 Declare Base Classifers we'll use for Building our Models

In [33]:
svmClassifier = SVC(kernel='linear',class_weight='balanced',probability=True)
lg = LogisticRegression()
dt = DecisionTreeClassifier()

In [36]:
#Support Vector Machine 
svmClassifier.fit(X_train,y_train)
svm_predict = svmClassifier.predict(X_test)
print("MCI-Controls(SVC Accuracy): %0.2f" % accuracy_score(y_test,svm_predict))

#Logistic Regression Classifer
lg.fit(X_train, y_train)
y_predict = lg.predict(X_test)
print("MCI-Controls(Logistic Regression Accuracy): %0.2f" % accuracy_score(y_test,y_predict))

#Decision Tree Classifer
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
print("MCI-Controls(Decision Trees Accuracy): %0.2f" % accuracy_score(y_test,y_predict))

MCI-Controls(SVC Accuracy): 1.00
MCI-Controls(Logistic Regression Accuracy): 1.00
MCI-Controls(Decision Trees Accuracy): 1.00


2. Using ADD Data

In [37]:
X_train, X_test, y_train, y_test = train_test_split(add_controls_features, add_controls_label, test_size=0.3, random_state=40)

In [38]:
X_train, X_test = scale_training_set(X_train, X_test)

/home/pesapal030/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [39]:
#Support Vector Machine 
svmClassifier.fit(X_train,y_train)
svm_predict = svmClassifier.predict(X_test)
print("ADD-Controls(SVC Accuracy): %0.2f " % accuracy_score(y_test,svm_predict))

#Logistic Regression
lg.fit(X_train, y_train)
y_predict = lg.predict(X_test)
print("ADD-Controls(Logistic Regression Accuracy): %0.2f" % accuracy_score(y_test,y_predict))


#Decision Tree Classifer
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
print("ADD-Controls(Decision Trees Accuracy): %0.2f " % accuracy_score(y_test,y_predict))

ADD-Controls(SVC Accuracy): 1.00 
ADD-Controls(Logistic Regression Accuracy): 1.00
ADD-Controls(Decision Trees Accuracy): 1.00 


3. Using the MCI -ADD -Controls Dataset

In [40]:
X_train, X_test, y_train, y_test = train_test_split(mci_add_controls_features, mci_add_controls_label, test_size=0.3, random_state=40)

In [41]:
X_train, X_test = scale_training_set(X_train, X_test)

/home/pesapal030/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [42]:
#Support Vector Machine 
svmClassifier.fit(X_train,y_train)
svm_predict = svmClassifier.predict(X_test)
print("MCI-ADD-Controls(SVC Accuracy): %0.2f" % accuracy_score(y_test,svm_predict))

#Logistic Regression
lg.fit(X_train, y_train)
y_predict = lg.predict(X_test)
print("MCI-ADD-Controls(Logistic Regression Accuracy): %0.2f" % accuracy_score(y_test,y_predict))


#Decision Tree Classifer
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
print("MCI-ADD-Controls(Decision Trees Accuracy): %0.2f" % accuracy_score(y_test,y_predict))


MCI-ADD-Controls(SVC Accuracy): 1.00
MCI-ADD-Controls(Logistic Regression Accuracy): 1.00
MCI-ADD-Controls(Decision Trees Accuracy): 1.00
